In [ ]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello

In [97]:
from bkmonitor.models.metric_list_cache import MetricListCache

groups = [
    {"data_source_label", "result_table_id"},
    {"data_source_label", "result_table_id", "bk_biz_id"},
    {"data_source_label", "result_table_id", "bk_biz_id", "data_type_label"},
    {"data_source_label", "result_table_id", "data_type_label"},
]
# 用于保存需要查询的参数
param_dics_list = []

for item in groups:
    result = MetricListCache.objects.all().values(*item).distinct()[0:10]
    param_dics_list.append(list(result))

In [ ]:
def query_metric_cache_loop(pre_field_set):
    result = []
    for param_dics in param_dics_list:
        for metric_info in param_dics:
            query_set = MetricListCache.objects.filter(**metric_info).order_by("id").values(*pre_field_set)
            for item in query_set:
                result.append(tuple(item[field] for field in pre_field_set))
    result.sort()
    return result

# pre_field_set = {"id",
#                  "result_table_id",
#                  "data_source_label",
#                  "metric_field",
#                  }
# 
# query_metric_cache_loop(pre_field_set)




In [ ]:
from functools import reduce
from itertools import count
from collections import defaultdict
import operator
from django.db.models import Q as DQ


def query_metric_cache_batch(pre_field_set):
    # 字段与目标值的映射
    field_values_map = defaultdict(list)
    # 过滤条件分组字典，过滤字段相同的为一个组
    filter_conditions_group = defaultdict(list)

    field_set = pre_field_set.copy()

    for param_dics in param_dics_list:
        for metric_info in param_dics:
            for key, value in metric_info.items():
                field_values_map[key].append(value)

            # 对过滤字段进行分组，并保存与当前metric_info相关的信息
            filter_conditions_group[tuple(sorted(metric_info.keys()))].append((metric_info))
            # 更新查询需要显示的字段集合
            field_set.update(set(metric_info.keys()))

    # 生成过滤条件
    filter_conditions = reduce(operator.or_, [DQ(**{key + "__in": field_values_map[key] for key in key_tuple})
                                              for key_tuple in filter_conditions_group.keys()])
    # print(filter_conditions)
    # 批量获取所有需要查询的指标
    metric_data_set = MetricListCache.objects.filter(filter_conditions).order_by("id").values(*field_set)
    # print("批量查询，数据量：" + str(len(metric_data_set)))
    result = []

    # print(filter_conditions_group)

    for key_tuple, met_info_recommends in filter_conditions_group.items():
        # old = len(result)
        metric_dic = defaultdict(list)

        # 根据key_tuple组成新的key，用于查询
        for data in metric_data_set:
            metric_dic[tuple(data[key] for key in key_tuple)].append(data)

        length = 0
        for item in metric_dic.values():
            length += len(item)

        # print(len(result), "当前数据量：", length)
        # k = count()
        for metric_info in met_info_recommends:
            metric_data = metric_dic.get(tuple(metric_info[key] for key in key_tuple))

            if not metric_data:
                # 没有获取到目标数据，跳过
                continue
            for item in metric_data:
                # next(k)
                metric = MetricListCache(**item)
                result.append(tuple(getattr(metric, field) for field in pre_field_set))
        # print(len(result) - old, key_tuple, next(k))

    result.sort()

    return result

# pre_field_set = {"id",
#                  "result_table_id",
#                  "data_source_label",
#                  "metric_field",
#                  }
# query_metric_cache_batch(pre_field_set)


In [142]:
# 测试使用pandas 进行过滤的数据与实际情况是否一致

pre_field_set = {"id",
                 "result_table_id",
                 "data_source_label",
                 "metric_field",
                 }

if __name__ == '__main__':
    # 循环获取并过滤数据    
    result_loop = query_metric_cache_loop(pre_field_set)
    # 批量获取数据，并使用pandas 进行过滤
    result_batch = query_metric_cache_batch(pre_field_set)
    print("两查询结果是否一致：", result_loop == result_batch)
    print(result_loop[0:10], len(result_loop))
    print()
    print(result_batch[0:20], len(result_batch))


908
量查询结果是否一致： True
[('124', 44387001, 'bk_monitor', 'strategy'), ('124', 44387001, 'bk_monitor', 'strategy'), ('125', 44387002, 'bk_monitor', 'strategy'), ('125', 44387002, 'bk_monitor', 'strategy'), ('128', 44387003, 'bk_monitor', 'strategy'), ('128', 44387003, 'bk_monitor', 'strategy'), ('129', 44387004, 'bk_monitor', 'strategy'), ('129', 44387004, 'bk_monitor', 'strategy'), ('135', 85, 'bk_monitor', 'strategy'), ('135', 85, 'bk_monitor', 'strategy')] 908

[('124', 44387001, 'bk_monitor', 'strategy'), ('124', 44387001, 'bk_monitor', 'strategy'), ('125', 44387002, 'bk_monitor', 'strategy'), ('125', 44387002, 'bk_monitor', 'strategy'), ('128', 44387003, 'bk_monitor', 'strategy'), ('128', 44387003, 'bk_monitor', 'strategy'), ('129', 44387004, 'bk_monitor', 'strategy'), ('129', 44387004, 'bk_monitor', 'strategy'), ('135', 85, 'bk_monitor', 'strategy'), ('135', 85, 'bk_monitor', 'strategy'), ('135', 85, 'bk_monitor', 'strategy'), ('135', 85, 'bk_monitor', 'strategy'), ('136', 44387005, '

In [ ]:
# 断点测试generate_recommended_metric_panels 方法
from fta_web.alert.resources import MetricRecommendationResource
from bkmonitor.aiops.alert.utils import RecommendMetricManager

params = [
    {'data_source_label': 'bk_monitor', 'result_table_id': 'strategy', 'data_type_label': 'alert', 'bk_biz_id': 17},
    {'data_source_label': 'bk_monitor', 'data_type_label': 'time_series'}
]
params = iter(params)

RecommendMetricManager(1, True).generate_recommended_metric_panels(1, {}, {})